# Exercises for Lecture 14 (Convolutional neural networks)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# To make this notebook's output stable across runs
def reset_state(seed=42):
    tf.keras.backend.clear_session()
    tf.random.set_seed(seed)
    np.random.seed(seed)

2022-02-14 21:51:24.751559: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-14 21:51:24.751595: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Exercise 1: Build a deeper CNN architecture for fashion MNIST than considered in the corresponding lecture to achieves a better classification accuracy.

Following the guiding CNN design principles covered in the lecture to construct a better CNN architecture.

Load and set up data.

In [2]:
# Load fashion MNIST data
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-30000], X_train_full[-30000:]
y_train, y_valid = y_train_full[:-30000], y_train_full[-30000:]

# Standardize
X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

# Add final channel axis (one channel)
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

Build and fit new model, and evaluate its performance.

In [3]:
model = keras.models.Sequential([
    
    keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu", padding="same", 
                        input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(pool_size=2),
    
    keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(pool_size=2),
    
    keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(pool_size=2),
    
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(units=10, activation='softmax'),
])

2022-02-14 21:51:27.501175: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-14 21:51:27.501211: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-14 21:51:27.501233: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fv-az75-959): /proc/driver/nvidia/version does not exist
2022-02-14 21:51:27.501547: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        3200      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 256)         295168    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 256)        0

In [5]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=3, validation_data=(X_valid, y_valid))
model.evaluate(X_test, y_test)

Epoch 1/3
938/938 [==============================] - 89s 93ms/step - loss: 0.8254 - accuracy: 0.7047 - val_loss: 0.4277 - val_accuracy: 0.8461
Epoch 2/3
938/938 [==============================] - 88s 94ms/step - loss: 0.4889 - accuracy: 0.8330 - val_loss: 0.3595 - val_accuracy: 0.8729
Epoch 3/3
313/313 [==============================] - 5s 15ms/step - loss: 0.3398 - accuracy: 0.8723


[0.33978885412216187, 0.8723000288009644]

Note that we have only trained for a small number of epochs.  

If you train for more epochs performance will hopefully improve.